# Interactive Map of Play Spaces in Houston, TX  

Playing outside, climbing, jumping, swinging and laughing is not only one of the most quintessential childhood activities, it is an indispensable need for children during their development - physically, cognitively and socially.  This is especially important today, with children spending less time engaged in outdoor play, and more time glued to screens.  Schools are another culprit - trading valuable time spent playing during recess for increased indoor classroom time in an effort to boost test-scores.  This strategy, although seemingly common sense, is counter to the findings of numerous research  studies showing increased academic performance and better behavioural outcomes in children who are provided with longer (30-60 minutes) recess breaks in the middle of the day.  

In addition, the concomitant rise in childhood obesity and development of near-sightedness along with decreasing time spent playing outside is alarming, but the solution is simple.  

![alt text](https://res.cloudinary.com/mommy-nearest/image/upload/c_fill,h_450,w_800/sjd19q4jlstuhywc50cy.jpg "Discovery Green")


### Increase outdoor playtime for children.  

One strategy would be to rally parents to petition their school boards and School Health Advisory Councils to increase recess for their children, and to protect this time from being reduced in favor of cramming for tests.  One children's advocacy group, [Children at Risk](https://childrenatrisk.org/wp-content/uploads/2018/07/CHILDREN-AT-RISK-School-Recess-Report.pdf), is working on just that.  Unfortunately, changing school policies is likely a very slow and tedious process, and parents may want to take measures into their own hands, and take their children to playgrounds, pools, water parks, and skating parks more often.  

The Houston Texas Parks and Recreation Department has a list of all public Houston play spaces with a small description of the kinds of equipment present.  I have created a simple interactive map to help parents find playgrounds, pools, water parks, and skating parks near them.

*For more information on the effects of outdoor playtime on children's health and performance, please see references at the bottom.*


In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import math
from time import sleep

#PART 1: Get the playground data from the Houston TX Parks & Recreation Department website
#Scraping tables from a website adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

def check_table(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    #Check the length of the first 12 rows
    return [len(T) for T in tr_elements[:12]]

def get_playgrounds(url, skip = 0):

    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    tr_elements = tr_elements[skip:]
    
    #Create empty list
    col=[]
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col.append((name,[]))
        
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=5:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
            
    #Create a dataframe        
    Dict={title:column for (title,column) in col}
    return pd.DataFrame(Dict)

#urls of playground pages
urls = ['https://www.houstontx.gov/parks/playgroundsA-F.html', 'https://www.houstontx.gov/parks/playgroundsG-N.html',
       'https://www.houstontx.gov/parks/playgroundsO-Z.html']
  
#Get playground tables into dataframes
play_1 = get_playgrounds(urls[0], skip = 1)
play_2 = get_playgrounds(urls[1], skip = 1)
play_3 = get_playgrounds(urls[2], skip = 1)

#Combine dataframes
playgrounds = pd.concat([play_1, play_2, play_3])
playgrounds.reset_index(drop = True, inplace = True)

In [2]:
##PART 2: Clean the data and gather geo data for mapping

#Detach Zipcodes from the Addresses
def detach_zip(df, address_column):
    for i in range(len(df[address_column])):
        if df.Address[i][-5:].isdigit():
            df.at[i,"Zipcode"] = df.at[i,address_column][-5:]
            df.at[i,address_column] = df.at[i,address_column][:-6]
            df.at[i,address_column].rstrip()
    return df

#Get the latitudes and longitudes of all the locations for mapping
def get_geodata(df):
    #Create columns for more data
    df["Zipcode"] = 0
    df["CityState"] = "Houston, TX"
    df["FullAddress"] = ""
    df["Latitude"] = np.nan
    df["Longitude"] = np.nan

    geolocator = Nominatim(user_agent="Houston_Playgrounds")

    for i in range(len(df.Address)):
        if math.isnan(df.Zipcode[i]):
            continue
        location = geolocator.geocode(df.Address[i]+" "+df.CityState[i])
        if location is not None:
            df.at[i,"FullAddress"] = location.address
            df.at[i,"Latitude"] = location.latitude
            df.at[i,"Longitude"] = location.longitude
        else:
            location = geolocator.geocode(df["Facility Name"][i]+", Houston, TX")
            if location is not None:
                df.at[i,"FullAddress"] = location.address
                df.at[i,"Latitude"] = location.latitude
                df.at[i,"Longitude"] = location.longitude
        sleep(0.10)
    return df

playgrounds = get_geodata(detach_zip(playgrounds, "Address"))
playgrounds.head(10)

,Facility Name,Address,Play Equip.,Swings,Other Equip.,Zipcode,CityState,FullAddress,Latitude,Longitude
0,Agnes Moffitt Park,10645 Hammerly Blvd.,X,,,0,"Houston, TX","10645, Hammerly Boulevard, Springwoods Village...",29.812595,-95.560035
1,Alief Park,11903 Bellaire Blvd.,X,,,0,"Houston, TX","11903, Bellaire Boulevard, Houston, Harris Cou...",29.703804,-95.588640
2,Almeda Park,14201 Almeda School Rd.,X,X,X,0,"Houston, TX","14201, Almeda School Road, Avondale, Houston, ...",29.600815,-95.417245
3,American Legion Park,3621 Golf Dr.,X,X,X,0,"Houston, TX","3621, Golf Drive, Shepherd Park Plaza, Houston...",29.819794,-95.424773
4,Andover Park,6301 Nunn St.,X,X,,0,"Houston, TX","6301, Nunn Street, Houston, Harris County, Tex...",29.673838,-95.310349
5,Aron Ledet Park,6323 Antoine @ Holly View,X,,,0,"Houston, TX","Aron Ledet Park, Near Northwest, Houston, Harr...",29.859112,-95.476391
6,Baldwin Park,1701 Elgin St.,X,X,X,0,"Houston, TX","1701, Elgin Street, Houston, Harris County, Te...",29.737999,-95.370859
7,Barbara Jordan Park,2400 Wipprecht St.,X,,,0,"Houston, TX","2400, Wipprecht Street, Houston, Harris County...",29.782076,-95.321866
8,Bayou Bend Park,3200 N MacGregor Way,X,X,X,0,"Houston, TX","3200, North MacGregor Way, Houston, Harris Cou...",29.712212,-95.369900
9,Beech-White Park,7551 Scott,X,,,0,"Houston, TX","Scott Street, Houston, Harris County, Texas, 7...",29.733680,-95.347981


In [4]:
#Import csv data about pools, waterparks and skating parks
pools = pd.read_csv('HoustonPools.csv', skiprows = 0, encoding='cp1252') 
waterparks = pd.read_csv('HoustonWaterparks.csv', skiprows = 0, encoding='cp1252') 
skating = pd.read_csv('HoustonSkateparks.csv', skiprows = 0, encoding='cp1252')

#Get geo data for pools, waterparks and skating parks
pools = get_geodata(detach_zip(pools, "Address"))
waterparks = get_geodata(waterparks)
skating = get_geodata(skating)

In [5]:
#Not all of the playgrounds and parks were able to be geolocated
#Grab all addresses that do not have latitudes and longitudes

def need_address(df):
    need = df.copy()
    need = (need
                    .where((need["Latitude"].isnull()== True) & (need["Address"].isnull() == False))
                    .dropna(subset = ["Address"]))
    need.reset_index(inplace = True, drop = True)
    return need

NA_play = need_address(playgrounds)
NA_pools = need_address(pools)
NA_waterparks = need_address(waterparks)
NA_skating = need_address(skating)

In [6]:
#PART 3: Create the map of playgrounds and parks

#Creates a folium feature group for the playgrounds
def playground_feature_group(df, colors, radius = 5):
    import folium
    
    playgrounds = df.copy()
    playgrounds.dropna(inplace = True)
    playgrounds.reset_index(inplace = True, drop = True)
    
    feature_group = FeatureGroup(name='Playgrounds')
    
    # for each row in the data, add a cicle marker
    for index, row in playgrounds.iterrows():
                
        # generate the popup message that is shown on click.
        popup_text = "<b>{}</b><br> {}<br><br> Play equipment: {}<br> Swings: {}<br> Other equipment: {}"
        popup_text = popup_text.format(row["Facility Name"],
                          row["Address"],
                          row["Play Equip."],
                          row["Swings"],
                          row["Other Equip."])
        
        # choose the color of the marker
        if row["Swings"] == "X":
            color= colors[0]
        else:     
            color=colors[1] 
        
        # add marker to the map
        feature_group.add_child(folium.CircleMarker(location=(row["Latitude"],
                                        row["Longitude"]),
                                        radius=radius,
                                        color=color,
                                        popup=popup_text,
                                        fill=True))
             
    return feature_group

#Creates a feature group for any other kind of park
def other_feature_group(df, name, colors, radius = 5):
    import folium
    
    places = df.copy()
    places.dropna(inplace = True)
    places.reset_index(inplace = True, drop = True)
    
    feature_group = FeatureGroup(name=name)
    
    # for each row in the data, add a cicle marker
    for index, row in places.iterrows():
                
        # generate the popup message that is shown on click.
        popup_text = "<b>{}</b><br> {}"
        popup_text = popup_text.format(row["Facility Name"],
                          row["Address"])
      
        # add marker to the map
        feature_group.add_child(folium.CircleMarker(location=(row["Latitude"],
                                        row["Longitude"]),
                                        radius=radius,
                                        color=colors,
                                        popup=popup_text,
                                        fill=True))
             
    return feature_group

In [7]:
#Creating the map with multiple layers

import folium
from folium.map import *

# generate a map of Houston
geolocator = Nominatim(user_agent="Houston_Playgrounds")
location = geolocator.geocode("Houston, TX")
folium_map = folium.Map(location=[location.latitude, location.longitude],
                        zoom_start=11,
                        tiles="CartoDB positron",
                        width='100%')

#Decide on colors for the various markers
colors = ["#F34526","#FFD735","#3551FF","#35EAFF","#6DF326"]

#Create different map feature groups for each park type
folium_map.add_child(playground_feature_group(playgrounds, colors = colors[:2]))
folium_map.add_child(other_feature_group(pools, name = "Pools", colors = colors[2]))
folium_map.add_child(other_feature_group(waterparks, name = "Water Parks", colors = colors[3]))
folium_map.add_child(other_feature_group(skating, name = "Skating Parks", colors = colors[4]))

folium_map.add_child(folium.map.LayerControl())
#folium_map


In [8]:
#Create a draggable legend for the map
#Solution adapted from http://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd

from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:16px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Houston Play Spaces</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#F34526;opacity:0.7;'></span>Playground</li>
    <li><span style='background:#FFD735;opacity:0.7;'></span>Playground with swings</li>
    <li><span style='background:#3551FF;opacity:0.7;'></span>Pool</li>
    <li><span style='background:#35EAFF;opacity:0.7;'></span>Water park</li>
    <li><span style='background:#6DF326;opacity:0.7;'></span>Skating park</li>
<div class = 'legend-footer'>Data from Houston Parks & Rec. Dept</div>
    
    
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-footer {
    text-align: left;
    margin-top: 5px;
    margin-bottom: 0px;
    font-size: 50%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

folium_map.get_root().add_child(macro)

In [9]:
# Saving the map to an html file
outfp = "Houston_playgrounds_map.html"
folium_map.save(outfp)

### References  

**Benefits of Recess:**  

*  American Academy of Pediatrics, “Policy Statement: The Crucial Role of Recess in School,” Pediatrics 131, no. 1 (2013): 186.  
*  Lloyd J. Kolbe, Laura Kann, Janet L. Collins, Meg Leavy Small, Beth Collins Pateman, and Charles W. Warren, “The School Health Policies and Programs Study (SHPPS): Context, Methods, General Findings, and Future Efforts,” Journal of School Health 65 (1995): 339.  
*  Promoting Better Health for Young People through Physical Activity and Sports (Washington, DC: U.S. Department of Health and Human Services and U.S. Department of Education), app. 7, www.thenewpe.com/advocacy/promotingPA.pdf.  
*  Olga S. Jarrett, Darlene M. Maxwell, Carrie Dickerson, Pamela Hoge, Gwen Davies, and Amy Yetley, “Impact of Recess on Classroom Behavior: Group Effects and Individual Differences,” Journal of Educational Research 92 (1998): 121–126.  
*  Centers for Disease Control and Prevention, “School Health Policies and Practices Study: 2014 Overview” (Atlanta: U.S. Department of Health and Human Services, 2015), 1.  
*  Romina M. Barros, Ellen J. Silver, and Ruth E. K. Stein, “School Recess and Group Classroom Behavior,” Pediatrics 123 (2009): 431–436.  
*  Centers for Disease Control and Prevention, “Guidelines for School and Community Programs to Promote Lifelong Physical Activity among Young People,” Morbidity and Mortality Weekly Report 46, no. RR-6 (March 7, 1997): 12.  
*  Deborah J. Rhea, Alexander P. Rivchun, and Jacqueline Pennings, “The Liink Project: Implementation of a Recess and Character Development Pilot Study with Grades K & 1 Children,” Texas Association for Health, Physical Education, Recreation & Dance Journal 84, no. 2 (Summer 2016): 14–17, 35.  
*  Nicola D. Ridgers, Gareth Stratton, Stuart J. Fairclough, and Jos W. R. Twisk, “Long-Term Effects of a Playground Markings and Physical Structures on Children’s Recess Physical Activity Levels,” Preventative Medicine 44 (2007): 393–397.  
*  Deborah J. Rhea and Irene Nigaglioni, “Outdoor Playing = Outdoor Learning,” Educational Facility Planner 49, nos. 2–3 (2016): 16–20.  
*  Lindsey Turner, Jamie F. Chriqui, and Frank J. Chaloupka, “Withholding Recess from Elementary School Students: Policies Matter,” Journal of School Health 83 (2013): 533–541.  

**Physical Activity and Academic Performance in Children:**  
*  Kamijo K, Pontifex MB, O’Leary KC, Scudder MR, Wu CT, Castelli DM, Hillman CH. The effects of an afterschool physical activity program on working memory in preadolescent children, Developmental Science. 2011; 14(5): 1046-1058.  
*  Pesce C, Crova C, Cereatti L, Casella R, Bellucci M. Physical activity and mental performance in preadolescents: effects of acute exercise on free-recall memory, Mental Health and Physical Activity. 2009. 2(1): 16-22.  
*  Strong WB, Malina RM, Blimkie CJR, Daniels SR, Dishman RK, Gutin B, Hergenroeder AC, Must A, Nixon PA, Pivarnik JM, Rowland T, Trost S, Trudeau F. Evidence based physical activity for school-age youth, The Journal of Pediatrics. 2005. 146(6): 732-737.  *  Sibley BA, Etnier JL, The relationship between physical activity and cognition in children: a meta-analysis, Pediatric Exercise Science. 2003. 15(3): 243-256.  
*  Aadland KN, Moe VF, Aadland E, Anderssen SA, Resaland GK, Ommundsen Y, Relationships between physical activity, sedentary time, aerobic fitness, motor skills and executive function and academic performance in children, Mental Health and Physical Activity. 2017. 12: 10-18.  
*  Haapala EA et al, Physical activity and sedentary time in relation to academic achievement in children, Journal of Science and Medicine in Sport. 2017. 20(6): 583-589.  
*  Kao SC, Westfall DR, Parks AC, Pontifex MB, Hillman CH. Muscular and aerobic fitness, working memory, and academic achievement in children, Medicine and Science in Sports and Exercise. 2016. 

**Myopia Development and Sunlight Exposure:**  
*  Rose KA, Morgan IG, Ip J, Kifley A, Huynh S, Smith W, et al. Outdoor activity reduces the prevalence of myopia in children. Ophthalmology. 2008;115(8):1279–85.  
*  He M, Xiang F, Zeng Y, Mai J, Chen Q, Zhang J, et al. Effect of time spent outdoors at school on the development of myopia among children in China: a randomized clinical trial. JAMA. 2015;314(11):1142–8.  
*  Wu PC, Tsai CL, Wu HL, Yang YH, Kuo HK. Outdoor activity during class recess reduces myopia onset and progression in school children. Ophthalmology. 2013;120(5): 1080–5.  
*  Yi JH, Li RR. Influence of near-work and outdoor activities on myopia progression in school children. Zhongguo Dang Dai Er Ke Za Zhi. 2011;13(1):32–5.  
*  Read SA, Collins MJ, Vincent SJ. Light Exposure and Eye Growth in Childhood. Invest Ophthalmol Vis Sci. 2015;56(11):6779–87.  